In [7]:
import pickle
from commonfunctions import *
import cv2

In [8]:
def img_pre_processing(image):

    # (1) crop background
    #cropped_image=crop_background(image)

    # (2) Apply Gaussian filtering to smooth out the image and remove any noise that may affect the accuracy of the HOG feature extractor
    #we need to try different kernel sizes for the best!
    gaussian_image = cv2.GaussianBlur(image, (3, 3), 0)

    # Convert the image to floating point
    gaussian_image_float = gaussian_image.astype(np.float32)

    # Calculate the histogram
    hist = cv2.calcHist([image], [0], None, [256], [0,256])

    # Calculate the mean of the histogram
    mean_hist = np.mean(hist)

    # Convert the image to HSV color space
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # Extract the S channel
    s_channel = hsv_image[:, :, 1]

    # Calculate the average saturation
    average_saturation = np.mean(s_channel)

    # Decide the gamma value
    if (average_saturation > 128) and (mean_hist > 128):
        gamma = 1.2  # or any value greater than 1
    else:
        gamma = 0.8  # or any value less than 1

    # Apply gamma correction
    gamma_corrected_image = cv2.pow(gaussian_image_float/ 255, gamma)


    final_image_before_HOG=gamma_corrected_image*255
    
    image=final_image_before_HOG.astype(np.uint8)

    
    return image

In [9]:
def HoG(image):
    # Resize the image to 130x276
    resized_img = cv2.resize(image, (64, 128))
    # print(resized_img.shape)

    # Convert the original image to gray scale
    resized_img_gray = cv2.cvtColor(resized_img, cv2.COLOR_BGR2GRAY)

    # Specify the parameters for our HOG descriptor
    #win_size = (64, 128)  # You need to set a proper window size
    #block_size = (16, 16)
    #block_stride = (8, 8)
    #cell_size = (8, 8)
    #num_bins = 9

    win_size = (64, 128)  # Increase the window size
    block_size = (32, 32)  # Increase the block size
    block_stride = (16, 16)  # Increase the block stride
    cell_size = (16, 16)  # Increase the cell size
    num_bins = 18  # Increase the number of bins

    # Set the parameters of the HOG descriptor using the variables defined above
    hog = cv2.HOGDescriptor(win_size, block_size, block_stride, cell_size, num_bins)

    # Compute the HOG Descriptor for the gray scale image
    hog_descriptor = hog.compute(resized_img_gray)
    return hog_descriptor

def hist_hsv(image):
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    #hist = cv2.calcHist([hsv_image], [0], None, [256], [0, 256])
    #hist /= hist.sum()

    # Get the H channel
    image1_h = hsv_image[:, :, 0]

    # Get the S channel
    image1_s = hsv_image[:, :, 1]

    # Calculate the histograms for S, H, and V channels
    hist_h = cv2.calcHist([image1_h], [0], None, [256], [0, 256])
    hist_s = cv2.calcHist([image1_s], [0], None, [256], [0, 256])
    hist_hs=cv2.calcHist([hsv_image[:,:,:1]],[0], None, [256], [0,256])


    # Normalize the histograms
    hist_h /= hist_h.sum()
    hist_s /= hist_s.sum()
    hist_hs /= hist_hs.sum()

    return hist_hs

In [10]:
#get pre trained model 
with open("Currency_model_prob.p", "rb") as model_file:
    trained_model = pickle.load(model_file)

with open("Saudi_model.p", "rb") as model_file:
    SAR_model = pickle.load(model_file)    

with open("Egypt_model.p", "rb") as model_file:
    EGP_model = pickle.load(model_file)

#with open("Error_handling_model.p", "rb") as model_file:
#    error_model = pickle.load(model_file)

In [11]:
img=io.imread("69.jpg")
image=img_pre_processing(img)

feat1=hist_hsv(img)
feat=HoG(image)
feat2=feat.reshape(1,-1)
feat1=feat1.reshape(1,-1)

In [12]:
classes=['1EGP','1SAR', '5EGP','5SAR','10EGP','10SAR','20EGP','50EGP','50SAR','100EGP','100SAR','200EGP','500SAR']
Egyptian=[0,2,4,6,7,9,11]
classes_EGP=['1EGP', '5EGP','10EGP','20EGP','50EGP','100EGP','200EGP']
Saudi=[1,3,5,8,10,12]
classes_SAR=['1SAR','5SAR','10SAR','50SAR','100SAR','500SAR']
#errors=['5SAR','10SAR','20EGP']
#make predictions
predictions = trained_model.predict(feat2)
predictions = int(predictions)
per=trained_model.predict_proba(feat2)
#write code to get max value of per
#maxval=np.amax(per,axis=1)
max_pred=(np.max(per))
#make max_pred into an float
max_pred=float(max_pred)
if (max_pred*100 >= 70):
    print(classes[predictions])
elif (max_pred*100 < 75  and max_pred*100 >= 40):
    if(predictions in Egyptian):
        predect= EGP_model.predict(feat2)
        per2=EGP_model.predict_proba(feat2)
        max_pred2=(np.max(per2))
        if (max_pred2*100 >= 75):
            predect = int(predect)
            print(classes_EGP[predect])
        else:
            print("Please enter another image")
    elif(predictions in Saudi):
        predect= SAR_model.predict(feat2)
        per2=SAR_model.predict_proba(feat2)
        max_pred2=(np.max(per2))
        if (max_pred2*100 >= 75):
            predect = int(predect)
            print(classes_SAR[predect])
        else:
            print("Please enter another image")        
else:
    print("please enter another image")
    

    '''
    eror_pred=error_model.predict(feat1)
    per2=error_model.predict_proba(feat1)
    max_pred2=(np.max(per2))
    #make max_pred into an float
    max_pred2=float(max_pred2)
    if (max_pred2*100 >= 55):
        eror_pred = int(eror_pred)
        print(errors[eror_pred])
    else:
    '''
    
#convert predictions variable to int
#predictions = int(predictions)
#type(predictions)
#print(classes[predictions])

please enter another image
